## CSV解析器

当您想要返回以逗号分隔的项目列表时，可以使用此输出解析器。

In [2]:
from langchain_openai import ChatOpenAI

api_key = "xxx"
base_url = "http://localhost:1234/v1"

chat = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.7)

chat.invoke("请问两只螃蟹多少条腿？")

AIMessage(content='两只螃蟹共有16条腿。因为每只螃蟹有8条腿，所以两只就是16条腿。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 36, 'total_tokens': 61, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5-32b-instruct', 'system_fingerprint': 'qwen2.5-32b-instruct', 'finish_reason': 'stop', 'logprobs': None}, id='run-86c0b5ca-855b-4b10-a362-b9bd1077ce2d-0', usage_metadata={'input_tokens': 36, 'output_tokens': 25, 'total_tokens': 61, 'input_token_details': {}, 'output_token_details': {}})

In [11]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser()

format_instructions = "您的输出应该是csv格式的逗号分隔符列表，例如： `内容1， 内容2， 内容3`"

chat_prompt = PromptTemplate(
    template="{format_instructions}\n请列出8种：{subject}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

chian = chat_prompt | chat | output_parser

chian.invoke({"subject": "川菜"})


['`宫保鸡丁， 麻婆豆腐， 水煮牛肉， 夫妻肺片， 回锅肉， 宫廷素烧海参， 口水鸡， 灯影牛肉干片`']

In [7]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [12]:
chat_prompt.format(subject="川菜")

'您的输出应该是csv格式的逗号分隔符列表，例如： `内容1， 内容2， 内容3`\n请列出8种：川菜'

## 日期时间解析器
此 OutputParser 可用于将 LLM 输出解析为日期时间格式。

In [16]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()

template = """
回答用户的问题：
{question}
{format_instructions}
"""

format_instructions = output_parser.get_format_instructions()
format_instructions = """
您的输出应该是日期时间格式，“%Y-%m-%dT%H:%M:%S.%fZ”。例如： 2024-01-01 12:00:00
"""

chat_prompt = PromptTemplate(
    template=template,
    partial_variables={"format_instructions": format_instructions}
)

chain = chat_prompt | chat | output_parser

chain.invoke({"question": "马斯克什么时间成立的特斯拉？"})


datetime.datetime(2003, 7, 1, 0, 0)

## Enum parser 枚举解析器

In [17]:
from langchain.output_parsers.enum import EnumOutputParser
from enum import Enum

class Colors(Enum):
    RED = "红色"
    BROWN = "棕色"
    BLACK = "黑色"
    WHITE = "白色"
    YELLOW = "黄色"
    

output_parser = EnumOutputParser(enum=Colors)


In [18]:
from langchain.prompts import PromptTemplate

chat_prompt = PromptTemplate.from_template(
    """
    {person}的皮肤是什么颜色，
    {instructions}
"""
)
instructions = "响应的结果请选择以下选项之一：红色、棕色、黑色、白色、黄色。不要有其他的内容"
chat_prompt = chat_prompt.partial(instructions=instructions)

chain = chat_prompt | chat | output_parser

In [19]:
instructions = output_parser.get_format_instructions()
instructions

'Select one of the following options: 红色, 棕色, 黑色, 白色, 黄色'

In [23]:
chain.invoke({"person": "非洲人"})

<Colors.BLACK: '黑色'>